In [46]:
#Este archivo es un webscrapping de la empresa de ropa HyM 

import requests
import re
import pandas as pd
import numpy as np
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
import pdb
from datetime import date
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.by import By


In [18]:
url = "https://co.hm.com/hombre/novedades"
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)

<ipython-input-18-28ad9d4125d9>:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


In [20]:
driver.get(url)
time.sleep(5)
products_button_path = './/a[@id="menu-product"]'
products_button = driver.find_element(By.XPATH, products_button_path)
driver.execute_script("arguments[0].click();", products_button)

In [56]:
grilla = driver.find_elements(By.XPATH, '//div[@id="gallery-layout-container"]')
n_item = 0
dict_item = {}
while True:
    try:
        # ================== Altura ================== #
        n_item += 1
        x_path = f'//div[@id="gallery-layout-container"]/div[{n_item}]'
        altura = driver.find_element("xpath", 
                    x_path).rect['y']
        driver.execute_script(f"window.scrollTo(0, {altura});")

        # ================== 1. item  ================== #
        path_item = f'{x_path}//a[@class = "vtex-product-summary-2-x-clearLink vtex-product-summary-2-x-clearLink--grilla h-100 flex flex-column"]'
        item = driver.find_element(By.XPATH, 
                    path_item).get_attribute('href')
        
        # ================== 2. Foto ================== #
        path_foto = f'{x_path}//div[@class="dib relative vtex-product-summary-2-x-imageContainer vtex-product-summary-2-x-imageContainer--commonImage vtex-product-summary-2-x-imageStackContainer vtex-product-summary-2-x-hoverEffect"]/img'
        foto = driver.find_element(By.XPATH, 
                    path_foto).get_attribute('src')

        # ================== 3. Nombre ================== #
        path_nombre = f'{x_path}//span[@class="vtex-product-summary-2-x-productBrand vtex-product-summary-2-x-brandName t-body"]'
        nombre = driver.find_element(By.XPATH, 
                    path_nombre).text     

        # ================== 4. Precio actual ================== #
        path_precio_actual_1 = f'{x_path}//span[@class="vtex-product-price-1-x-currencyInteger vtex-product-price-1-x-currencyInteger--newlistPrice"][1]'
        precio_actual_1 = driver.find_element(By.XPATH, 
                    path_precio_actual_1).text

        path_precio_actual_2 = f'{x_path}//span[@class="vtex-product-price-1-x-currencyInteger vtex-product-price-1-x-currencyInteger--newlistPrice"][2]'
        precio_actual_2 = driver.find_element(By.XPATH, 
                    path_precio_actual_2).text

        try:
            # ================== 5. Precio anterior ================== #
            path_precio_anterior_1 = f'{x_path}//span[@class="vtex-product-price-1-x-currencyInteger vtex-product-price-1-x-currencyInteger--bestPrice"][1]'
            precio_anterior_1 = driver.find_element(By.XPATH, 
                    path_precio_anterior_1).text

            path_precio_anterior_2 = f'{x_path}//span[@class="vtex-product-price-1-x-currencyInteger vtex-product-price-1-x-currencyInteger--bestPrice"][2]'
            precio_anterior_2 = driver.find_element(By.XPATH, 
                    path_precio_anterior_2).text

            # ================== 6. Descuento ================== #
            path_descuento = f'{x_path}//p[@class="hmchile-catalog-discount-0-x-discount_grilla"]'
            descuento = driver.find_element(By.XPATH, 
                    path_descuento).text  
        except: 
            precio_anterior_1 = 0
            precio_anterior_2 = 0
            descuento = 0
        

        dict_item[n_item] = {
            'fecha': date.today(),
            'foto': foto,
            'item': nombre,
            'posicion': n_item,
            'pag': url,
            'precio_actual': f'{precio_actual_1}{precio_actual_2}',
            'precio_anterior': f'{precio_anterior_1}{precio_anterior_2}',
            'genero': 'Hombre',
            'categoria': 'Novedades',
            'link': item,
            'descuento': descuento
        }


    except:
        try:
            load_more_products = './/div[@class="vtex-button__label flex items-center justify-center h-100 ph5 "]'
            load_more_products_button = driver.find_element(By.XPATH, 
                                    load_more_products)
            driver.execute_script("arguments[0].click();", 
                                    load_more_products_button)
            time.sleep(3.5)
            
        except Exception as e: 
            print(e)
            break

No hay mas productos
